<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/%E6%89%B9%E9%87%8F%E7%BF%BB%E8%AF%91%E4%B8%AD%E5%88%B0%E6%97%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [2]:
import os
import httpx

os.environ["OPENAI_API_KEY"]  = "sk-JEBR"

import openai
from openai import AsyncOpenAI

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls /content/drive/MyDrive/CardBuild/exp0131/input

source_0.jsonl	source_1.jsonl	source_2.jsonl	source_3.jsonl	source_4.jsonl


读取 /content/drive/MyDrive/CardBuild/exp0131/input 中所有jsonl文件

保存到datas中，给我python代码的实现

In [5]:
import os
import json

directory = '/content/drive/MyDrive/CardBuild/exp0131/input'
datas = []

# 列出指定目录中的所有文件
for filename in os.listdir(directory):
    # 检查文件扩展名是否为.jsonl
    if filename.endswith('.jsonl'):
        file_path = os.path.join(directory, filename)
        # 读取和解析.jsonl文件
        with open(file_path, 'r') as file:
            for line in file:
                # 将每行解析为JSON
                json_data = json.loads(line)
                datas.append(json_data)

# datas现在包含了所有jsonl文件中的数据


In [6]:
print(len(datas))

21400


In [8]:
translation_prompt = (
    "あなたは日本語に精通しているプロの翻訳者で、特に中国語の小説を原作者のスタイルを保ちつつ、わかりやすい日本語に翻訳することに長けています。以下の中国語の段落を日本語に翻訳し、中国語の原著に似たスタイルでお願いします。"
    "ルール："
    "- 翻訳する際は、原文の事実や背景を正確に伝える必要があります。"
    "- 意訳を行う場合でも、元の段落の形式を保ち、専門用語（例：FLAC、JPEGなど）もそのままにします。企業の略称もそのままにします（例：Microsoft、Amazon、OpenAIなど）。"
    "- 人名は翻訳しません"
    "戦略："
    "翻訳作業を以下の三つのステップで行い、各ステップの結果を出力します："
    "1. 中国語から日本語への直訳で、元の形式を保ち、どんな情報も省略せずに行います。"
    "2. 第一ステップの直訳の結果に基づき、具体的な問題点を指摘します。具体的に説明し、一般的な表現を避け、原文に存在しない内容や形式を加える必要はありません。問題には以下が含まれます："
    "  - 日本語の表現に合っていない点を明確に指摘します。"
    "  - 文章が不自然な箇所を指摘します。修正案は不要で、意訳で修正します。"
    "  - 難解で理解しにくい点について、説明を試みることができます。"
    "3. 第一ステップの直訳と第二ステップの問題点に基づき、中国語から日本語への意訳を再度行います。内容の本意を保ちつつ、より理解しやすく、日本語の表現に合ったものにします。元の形式はそのままにします。"
    "結果はjson形式で返します。例えば以下のように、実際の出力ではシングルクォート（\'）ではなくダブルクォート（\"）を使用します："
    "{"
    "    \"直訳結果\": \"***\","
    "    \"直訳の具体的な問題リスト\": \"***\","
    "    \"意訳結果\": \"***\","
    "}"
    "意訳結果の終わりには常にダブルクォート（\"）を使用してください。それでは、上記の要件に従って中国語を日本語に翻訳し、出力言語が日本語であることを確認してください。"
    )

In [9]:
print(translation_prompt)

あなたは日本語に精通しているプロの翻訳者で、特に中国語の小説を原作者のスタイルを保ちつつ、わかりやすい日本語に翻訳することに長けています。以下の中国語の段落を日本語に翻訳し、中国語の原著に似たスタイルでお願いします。ルール：- 翻訳する際は、原文の事実や背景を正確に伝える必要があります。- 意訳を行う場合でも、元の段落の形式を保ち、専門用語（例：FLAC、JPEGなど）もそのままにします。企業の略称もそのままにします（例：Microsoft、Amazon、OpenAIなど）。- 人名は翻訳しません戦略：翻訳作業を以下の三つのステップで行い、各ステップの結果を出力します：1. 中国語から日本語への直訳で、元の形式を保ち、どんな情報も省略せずに行います。2. 第一ステップの直訳の結果に基づき、具体的な問題点を指摘します。具体的に説明し、一般的な表現を避け、原文に存在しない内容や形式を加える必要はありません。問題には以下が含まれます：  - 日本語の表現に合っていない点を明確に指摘します。  - 文章が不自然な箇所を指摘します。修正案は不要で、意訳で修正します。  - 難解で理解しにくい点について、説明を試みることができます。3. 第一ステップの直訳と第二ステップの問題点に基づき、中国語から日本語への意訳を再度行います。内容の本意を保ちつつ、より理解しやすく、日本語の表現に合ったものにします。元の形式はそのままにします。結果はjson形式で返します。例えば以下のように、実際の出力ではシングルクォート（'）ではなくダブルクォート（"）を使用します：{    "直訳結果": "***",    "直訳の具体的な問題リスト": "***",    "意訳結果": "***",}意訳結果の終わりには常にダブルクォート（"）を使用してください。それでは、上記の要件に従って中国語を日本語に翻訳し、出力言語が日本語であることを確認してください。


In [10]:
print(datas[0].keys())

dict_keys(['from', 'value'])


In [12]:
query_datas = []

for i, data in enumerate( datas ):
    text = data["value"]
    messages = [
        {"role":"system", "content":translation_prompt},
        {"role":"user","content":text}
    ]
    query_data = {
        "id": str(i),
        "messages":messages
    }
    query_datas.append(query_data)

In [13]:
!pip install -q aiofiles tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [14]:
import os
import json
import time
# import openai
from openai import AsyncOpenAI

aclient = AsyncOpenAI()

import asyncio
import aiofiles
import tiktoken
import hashlib
# from connector import AsyncPGConnector
from tqdm.asyncio import tqdm as tqdm

enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
en2zh_ratio = 2.3

delay = 1
concurrency_limit = 16

max_file_size = 1024**3

In [20]:
async def dealing_messages(messages):
    try:
        # request_token = sum([len(enc.encode(msg['content'])) for msg in messages])
        # response_token = int(len(enc.encode(text)) * en2zh_ratio) + 64

        model = "gpt-3.5-turbo-1106"

        resp = await aclient.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0,
            response_format={ "type": "json_object" }
        )

        try:
            result = resp.choices[0].message.content
            result = result.strip()
            return result
        except json.JSONDecodeError:
            print("Invalid json: ", result)
            return None
        except:
            raise Exception(f"Invalid API response: {resp}")

    except Exception as e:
        print(f"[Error] {e}")
        return None


In [21]:
async def getTranslation(item):
    if "messages" not in item:
        return None
    else:
        for i in range(3):
            result = await dealing_messages(item['messages'])
            if result is not None:
                item["response"] = result
                return item
            else:
                continue

    return None

async def process(item, semaphore):
    async with semaphore:
        try:
            output_folder = "/content/output"
            if "output_folder" in item:
                output_folder = item["output_folder"]
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            file_path = os.path.join(output_folder, f"{item['id']}.txt")

            if os.path.exists(file_path):
                return

            it = await getTranslation(item)
            if it is None:
                raise Exception(item['id'])

            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(it, f, ensure_ascii=False)

        except Exception as e:
            print(f"Error processing entry: {e}")

In [22]:
async def main(datas):

    output_folder = "/content/output"

    process_data = []

    for data in datas:
        id = data['id']
        process_data.append({
            "id": id,
            "messages": data["messages"],
            "output_folder": output_folder
        })

    tasks = []

    semaphore = asyncio.Semaphore(concurrency_limit)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # # print(f"Already processed {len(exist_list)} items...")

    # id = set()

    for item in process_data:
        file_path = os.path.join(output_folder, f"{item['id']}.txt")

        if os.path.exists(file_path):
            continue

        tasks.append(asyncio.create_task(process(item, semaphore)))

    async for task in tqdm(tasks, total=len(tasks), desc="Processing items"):
        await task
        time.sleep(delay)



In [23]:
datas = query_datas

In [19]:
start_id = 0
end_id = 10

current_tasks = datas[start_id:end_id]

await main(current_tasks)

temp_output_folder = "/content/output"

for id in range(start_id, end_id):
    id_str = datas[id]["id"]
    file_path = os.path.join(temp_output_folder, f"{id_str}.txt")
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                response = data["response"]
            except:
                continue

        if response is not None:
            datas[id]["response"] = response
    # break

final_save_name = "/content/drive/MyDrive/CardBuild/exp0131/output/" + str(start_id) + "_to_" + str(end_id) + ".txt"

with open(final_save_name, 'w', encoding='utf-8') as f:
    for id in range(start_id, end_id):
        json.dump(datas[id], f, ensure_ascii=False)
        f.write('\n')

Processing items: 100%|██████████| 10/10 [00:44<00:00,  4.45s/it]


In [26]:
super_start = 10
super_end = len(datas)
n_bag = 50

# 计算每个子区间的长度
interval_length = (super_end - super_start) // n_bag

# 生成元组列表
se_tuples_list = [(super_start + i * interval_length, super_start + (i + 1) * interval_length) for i in range(n_bag - 1)]

# 添加最后一个元组，确保最后一个end等于super_end
se_tuples_list.append((se_tuples_list[-1][1], super_end))

# 打印生成的元组列表
print(se_tuples_list[0:2])
print(se_tuples_list[-2:])




[(10, 437), (437, 864)]
[(20506, 20933), (20933, 21400)]


In [ ]:
for start_id, end_id in se_tuples_list:
    current_tasks = datas[start_id:end_id]

    final_save_name = "/content/drive/MyDrive/CardBuild/exp0131/output/" + str(start_id) + "_to_" + str(end_id) + ".txt"

    if os.path.exists(final_save_name):
        continue

    await main(current_tasks)

    temp_output_folder = "/content/output"

    for id in range(start_id, end_id):
        id_str = datas[id]["id"]
        file_path = os.path.join(temp_output_folder, f"{id_str}.txt")
        if os.path.exists(file_path):
            with open(file_path, 'r', encoding='utf-8') as f:
                try:
                    data = json.load(f)
                    response = data["response"]
                except:
                    continue

            if response is not None:
                datas[id]["response"] = response
        # break



    with open(final_save_name, 'w', encoding='utf-8') as f:
        for id in range(start_id, end_id):
            json.dump(datas[id], f, ensure_ascii=False)
            f.write('\n')

Processing items:   5%|▌         | 20/382 [00:48<06:12,  1.03s/it]